# Rules of persona
+ Each sentence must contain between 4 and 20 words or punctuation marks.
+ It contains either the word I or my.
+ At least one verb, and (iv) at least one noun, pronoun or adjective.

# Example Dialogue with persona
- Persona: [“I like sport”, “I work a lot”]
- Context: “I love running.”
- Response: “Me too! But only on weekends.”

{
    
    "dialog": [["没有 钱   万万 不行 ！ ~"], ["现实 就是 如此"]], 
    
    "profile": [{"tag": ["漫画;旅遊;星座"], "loc": "广东 广州", "gender": "male"}, {"tag": [""], "loc": "", "gender": ""}], 
    
    "uid": [0, 1]

}

{
    
    "dialog": [["For what it's worth, I don't have a problem with it."], ["My apologies.  I did not have any problems with it, but I will be more careful in the future."]], 
    
    "profile": [{"tag": [], "loc": "", "gender": ""}, {"tag": ["i did not have any problems with it, but i will be more careful in the future."], 
    
    "loc": "", "gender": ""}], 
    
    "uid": [0, 1]

}

# test_data Example
{"uid": [0, 1, 2], "dialog": [["剧烈运动 是 吧"], ["各种 剧烈运动"], ["... 姐 最近 有点 寂寞 过头 了 ..."]], "responder_profile": {"loc": "海南", "gender": "female", "tag": "美食;宅;80后"}, "profile": [{"loc": "天津 滨海新区", "gender": "male", "tag": ""}, {"loc": "海南", "gender": "female", "tag": "美食;宅;80后"}, {"loc": "安徽 合肥", "gender": "male", "tag": "游戏动漫;双子座;宅;音乐;90后;WOW台服众"}], "golden_response": ["可不是 ， 我 又 不 像 你 ， 有 女神 。"]}

# Constant Value

In [245]:
NPARTITIONS = 1000
PATH = "./reddit_data/*/*.json"
SCHEDULER = "threads"

# Imports

In [246]:
import pandas as pd 
import json
import bz2
from tqdm import tqdm
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import spacy
import os
import redditcleaner
import neuralcoref

In [247]:
tqdm.pandas()
ProgressBar().register()
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

#doc1 = nlp('My sister has a dog. She loves him.')
#print(doc1._.coref_resolved)

In [248]:
if(not os.path.exists("./outputs")):
    os.makedirs("./outputs/")


In [249]:
version = len([f for f in os.listdir("./outputs") if "persona" in f])
version

1

# reddit_data下の全てのjsonファイルを読み込む

In [266]:
list_bz2_file = glob.glob(PATH)
list_reddit_conversation = []
list_bz2_file

['./reddit_data\\2007\\RC_2007-10.json',
 './reddit_data\\2008\\RC_2008-01.json']

In [267]:
print("----------read input json files----------")
for i in tqdm(range(0,len(list_bz2_file))):
    with open(list_bz2_file[i], mode="r", encoding="shift-jis") as f:
        for line in f.readlines():
            dic=json.loads(line)
            list_reddit_conversation.append(dic)

100%|██████████| 2/2 [00:00<00:00, 11.42it/s]

----------read input json files----------


In [268]:
df_reddit_conversation = pd.DataFrame(list_reddit_conversation)
df_reddit_conversation = df_reddit_conversation[df_reddit_conversation["author"]!="[deleted]"]
df_reddit_conversation["body"] = df_reddit_conversation["body"].progress_map(lambda x:redditcleaner.clean(str(x)))
df_reddit_conversation["body"] = df_reddit_conversation["body"].replace(["&lt","&gt","&amp"],["","",""])
df_reddit_conversation.to_csv(f"./outputs/AllConversation{version}.csv")
df_reddit_conversation.head(5)

100%|██████████| 13373/13373 [00:00<00:00, 40491.87it/s]


,parent_id,created_utc,ups,controversiality,distinguished,subreddit_id,id,downs,archived,link_id,...,author,score_hidden,body,gilded,author_flair_text,subreddit,edited,author_flair_css_class,name,retrieved_on
0,t3_5yba3,1192450635,1,0,None,t5_6,c0299an,0,True,t3_5yba3,...,bostich,False,test,0,None,reddit.com,False,None,t1_c0299an,1427426409
1,t3_5yba3,1192450639,2,0,None,t5_6,c0299ao,0,True,t3_5yba3,...,igiveyoumylife,False,"much smoother. Im just glad reddit is back, re...",0,None,reddit.com,False,None,t1_c0299ao,1427426409
2,t1_c02999p,1192450643,0,0,None,t5_6,c0299ap,0,True,t3_5yba3,...,Arve,False,"Can we please deprecate the word ""Ajax"" now? (...",0,None,reddit.com,False,None,t1_c0299ap,1427426409
4,t1_c0299ah,1192450646,3,0,None,t5_6,c0299ar,0,True,t3_5yba3,...,gigaquack,False,"Oh, I see. Fancy schmancy ""submitting....""",0,None,reddit.com,False,None,t1_c0299ar,1427426409
5,t3_5yba3,1192450656,1,0,None,t5_6,c0299as,0,True,t3_5yba3,...,Percept,False,testing ...,0,None,reddit.com,False,None,t1_c0299as,1427426409


In [ ]:
body="Hmm, ok.\n\nBBC news posted Dec 29, 15:17 GMT.\n\nshabby47 posted his answer \u20192 days ago.\u2019\n\n"

16003    Hmm, ok. BBC news posted Dec 29, 15:17 GMT. sh...
Name: body, dtype: object

In [ ]:
redditcleaner.clean(body)

In [ ]:
df_reddit_conversation[df_reddit_conversation["body"].str.contains("Hmm, ok.")]["body"]

# 会話ペアの作成

In [271]:
df_reddit_conversation = pd.DataFrame(list_reddit_conversation)
df_reddit_conversation = df_reddit_conversation[df_reddit_conversation["body"]!="[deleted]"]
df_reddit_conversation["removed_prefix_parent_id"] = df_reddit_conversation["parent_id"].str.replace("t\d_","")
df_reddit_conversation["parent_body"] = df_reddit_conversation[df_reddit_conversation["removed_prefix_parent_id"]==df_reddit_conversation["id"]]["body"]
df_reddit_conversation["body"] = df_reddit_conversation["body"].str.replace('\"','’')
df_reddit_conversation["parent_body"] = df_reddit_conversation["parent_body"].str.replace('\"','’')
df_reddit_conversation = pd.merge(df_reddit_conversation,df_reddit_conversation[["id","body"]].rename(columns={"id":"parent_id","body":"parent_body"}),left_on="removed_prefix_parent_id",right_on="parent_id").drop(columns=["parent_body_x","parent_id_y"]).rename(columns={"parent_body_y":"parent_body"})
df_reddit_conversation = df_reddit_conversation.dropna(subset=["parent_body"]).sort_values(["author"]).reset_index(drop=True)
df_reddit_conversation["original_body"] = df_reddit_conversation["body"]
df_reddit_conversation["original_parent_body"] = df_reddit_conversation["parent_body"]
df_reddit_conversation = df_reddit_conversation[["body","parent_body","original_body","original_parent_body","ups","author"]]
df_reddit_conversation

<ipython-input-271-9c3c4850382b>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_reddit_conversation["removed_prefix_parent_id"] = df_reddit_conversation["parent_id"].str.replace("t\d_","")


,body,parent_body,original_body,original_parent_body,ups,author
0,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,2,0gleth0rpe
1,Like hormonal imbalances and cancer?,Has anyone ever considered that perhaps the pl...,Like hormonal imbalances and cancer?,Has anyone ever considered that perhaps the pl...,1,24sparky
2,Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,1,24sparky
3,Happy New Year :),"Thanks for the links, im looking through them ...",Happy New Year :),"Thanks for the links, im looking through them ...",1,24sparky
4,"Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,"Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,1,24sparky
...,...,...,...,...,...,...
7843,Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83
7844,http://www.fff.org/comment/com0311c.asp\n\nmay...,Wow! What a shocker! After Saddam attempted to...,http://www.fff.org/comment/com0311c.asp\n\nmay...,Wow! What a shocker! After Saddam attempted to...,1,zorno
7845,"Yeah ok, but what I do is just go rapidly over...",I like it because the old system lent itself t...,"Yeah ok, but what I do is just go rapidly over...",I like it because the old system lent itself t...,1,zouhair
7846,’social’ added to ’problems’ do not have the s...,"Oh okay, you're right, let's just cancel all s...",’social’ added to ’problems’ do not have the s...,"Oh okay, you're right, let's just cancel all s...",2,zouhair


In [272]:
def CreatePersona(body: str):
    doc = nlp(body.lower())
    # 文ごとに分割
    persona = [str(sentence) for sentence in doc.sents if IsPersona(str(sentence))]
    return persona

In [273]:
def IsPersona(sentence: str):
    # 以下の3つの条件を満たすものをペルソナとする
    # 1.文の単語数が4-20の間
    # 2.I か my　が含まれている
    # 3.少なくとも1つの動詞と，名詞，代名詞，形容詞のいずれかが含まれている
    words = [str(word) for word in nlp(sentence.strip())]
    poses = [token.pos_ for token in nlp(sentence.strip())]
    return (
        (4 <= len(words) <= 20)&
        (not set(["i","my"]).isdisjoint(set(words)))&
        (("VERB" in poses)&(not set(["NOUN", "ADJ", "PROPN"]).isdisjoint(set(poses))))
    )

In [274]:
def create_json(row):
    return {
        "dialog":row["dialog"],
        "profile":[
            {"tag":row["persona"],
            "loc":"",
            "gender":""},
            {"tag":row["parent_persona"],
            "loc":"",
            "gender":""}
        ],
        "uid":[0,1]
    }

# ペルソナの作成

In [276]:
print("----------create conversation pair ----------")
ddf_reddit_conversation = dd.from_pandas(data=df_reddit_conversation, npartitions=NPARTITIONS)
ddf_reddit_conversation["persona"] = ddf_reddit_conversation["original_body"].map(CreatePersona)
ddf_reddit_conversation["parent_persona"] = ddf_reddit_conversation["original_parent_body"].map(CreatePersona)
ddf_reddit_conversation.query("persona.notnull() | parent_persona.notnull()")
#ddf_reddit_conversation["body"] = ddf_reddit_conversation["body"].map(lambda sentence:nlp(sentence)._.coref_resolved)
#ddf_reddit_conversation["parent_body"] = ddf_reddit_conversation["parent_body"].map(lambda sentence:nlp(sentence)._.coref_resolved)
df_reddit_conversation = ddf_reddit_conversation.compute(scheduler=SCHEDULER)

----------create conversation pair ----------
[####################################### ] | 99% Completed | 18min 27.6s

In [ ]:
print("--------- create list ----------")
df_reddit_conversation["body"] = df_reddit_conversation["body"].progress_map(lambda x: [x] )
df_reddit_conversation["parent_body"] = df_reddit_conversation["parent_body"].progress_map(lambda x: [x] )
df_reddit_conversation["dialog"] = [list(x) for x in zip(df_reddit_conversation["body"].tolist(),df_reddit_conversation["parent_body"].tolist())]
df_reddit_conversation

100%|██████████| 7848/7848 [00:00<00:00, 713351.63it/s]

--------- create list ----------


,body,parent_body,original_body,original_parent_body,ups,author,persona,parent_persona,dialog
0,"[For what it's worth, I don't have a problem w...",[My apologies. I did not have any problems wi...,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,2,0gleth0rpe,[],"[i did not have any problems with it, but i wi...","[[For what it's worth, I don't have a problem ..."
1,[Like hormonal imbalances and cancer? ],[Has anyone ever considered that perhaps the p...,Like hormonal imbalances and cancer?,Has anyone ever considered that perhaps the pl...,1,24sparky,[],[],"[[Like hormonal imbalances and cancer? ], [Has..."
2,[Yeah I wish I could get the test some attenti...,[I took that 6 page survey and ended up where ...,Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,1,24sparky,[i wish i could get the test some attention be...,[i took that 6 page survey and ended up where ...,[[Yeah I wish I could get the test some attent...
3,[Happy New Year :)],"[Thanks for the links, im looking through them...",Happy New Year :),"Thanks for the links, im looking through them ...",1,24sparky,[],"[thanks for the links, im looking through them...","[[Happy New Year :)], [Thanks for the links, i..."
4,"[Hmm, I came out as ’centrist’ but right betwe...",[World's Smallest Political Quiz:\r\nhttp://ww...,"Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,1,24sparky,"[hmm, i came out as ’centrist’ but right betwe...",[personally i think nolan chart is more useful...,"[[Hmm, I came out as ’centrist’ but right betw..."
...,...,...,...,...,...,...,...,...,...
7843,[Thanks for that- I always assumed the kinetic...,"[Not that it changes your point, but I think y...",Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83,[],"[not that it changes your point, but i think y...",[[Thanks for that- I always assumed the kineti...
7844,[http://www.fff.org/comment/com0311c.asp\n\nma...,[Wow! What a shocker! After Saddam attempted t...,http://www.fff.org/comment/com0311c.asp\n\nmay...,Wow! What a shocker! After Saddam attempted to...,1,zorno,[],[],[[http://www.fff.org/comment/com0311c.asp\n\nm...
7845,"[Yeah ok, but what I do is just go rapidly ove...",[I like it because the old system lent itself ...,"Yeah ok, but what I do is just go rapidly over...",I like it because the old system lent itself t...,1,zouhair,[],[],"[[Yeah ok, but what I do is just go rapidly ov..."
7846,[’social’ added to ’problems’ do not have the ...,"[Oh okay, you're right, let's just cancel all ...",’social’ added to ’problems’ do not have the s...,"Oh okay, you're right, let's just cancel all s...",2,zouhair,[],[],[[’social’ added to ’problems’ do not have the...


# Json形式の作成

In [ ]:
df_reddit_conversation["json"] = df_reddit_conversation.progress_apply(create_json, axis=1)
df_reddit_conversation

100%|██████████| 7848/7848 [00:00<00:00, 60338.97it/s]


,body,parent_body,original_body,original_parent_body,ups,author,persona,parent_persona,dialog,json
0,"[For what it's worth, I don't have a problem w...",[My apologies. I did not have any problems wi...,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,2,0gleth0rpe,[],"[i did not have any problems with it, but i wi...","[[For what it's worth, I don't have a problem ...","{'dialog': [['For what it's worth, I don't hav..."
1,[Like hormonal imbalances and cancer? ],[Has anyone ever considered that perhaps the p...,Like hormonal imbalances and cancer?,Has anyone ever considered that perhaps the pl...,1,24sparky,[],[],"[[Like hormonal imbalances and cancer? ], [Has...",{'dialog': [['Like hormonal imbalances and can...
2,[Yeah I wish I could get the test some attenti...,[I took that 6 page survey and ended up where ...,Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,1,24sparky,[i wish i could get the test some attention be...,[i took that 6 page survey and ended up where ...,[[Yeah I wish I could get the test some attent...,{'dialog': [['Yeah I wish I could get the test...
3,[Happy New Year :)],"[Thanks for the links, im looking through them...",Happy New Year :),"Thanks for the links, im looking through them ...",1,24sparky,[],"[thanks for the links, im looking through them...","[[Happy New Year :)], [Thanks for the links, i...","{'dialog': [['Happy New Year :)'], ['Thanks fo..."
4,"[Hmm, I came out as ’centrist’ but right betwe...",[World's Smallest Political Quiz:\r\nhttp://ww...,"Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,1,24sparky,"[hmm, i came out as ’centrist’ but right betwe...",[personally i think nolan chart is more useful...,"[[Hmm, I came out as ’centrist’ but right betw...","{'dialog': [['Hmm, I came out as ’centrist’ bu..."
...,...,...,...,...,...,...,...,...,...,...
7843,[Thanks for that- I always assumed the kinetic...,"[Not that it changes your point, but I think y...",Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83,[],"[not that it changes your point, but i think y...",[[Thanks for that- I always assumed the kineti...,{'dialog': [['Thanks for that- I always assume...
7844,[http://www.fff.org/comment/com0311c.asp\n\nma...,[Wow! What a shocker! After Saddam attempted t...,http://www.fff.org/comment/com0311c.asp\n\nmay...,Wow! What a shocker! After Saddam attempted to...,1,zorno,[],[],[[http://www.fff.org/comment/com0311c.asp\n\nm...,{'dialog': [['http://www.fff.org/comment/com03...
7845,"[Yeah ok, but what I do is just go rapidly ove...",[I like it because the old system lent itself ...,"Yeah ok, but what I do is just go rapidly over...",I like it because the old system lent itself t...,1,zouhair,[],[],"[[Yeah ok, but what I do is just go rapidly ov...","{'dialog': [['Yeah ok, but what I do is just g..."
7846,[’social’ added to ’problems’ do not have the ...,"[Oh okay, you're right, let's just cancel all ...",’social’ added to ’problems’ do not have the s...,"Oh okay, you're right, let's just cancel all s...",2,zouhair,[],[],[[’social’ added to ’problems’ do not have the...,{'dialog': [['’social’ added to ’problems’ do ...


# Outputs

In [ ]:
df_reddit_conversation.to_csv(f"./outputs/persona{version}.csv")

In [ ]:
list_json = df_reddit_conversation["json"].tolist()
with open(f"./outputs/created_dialogues{version}.json", "wt", encoding="utf-8") as file:
    for dic in list_json:
        file.write(str(json.dumps(dic))+"\n")

In [ ]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'], returncode=0)